<a href="https://colab.research.google.com/github/dmkolesova/SE_SQL/blob/main/%22DB_INSERT_ANY_TYPE_ipynb%22.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Код для генерации данных PostgreSQL

Инструкция по генерации строк:
1. Все ячейки кода необходимо запустить в той последовательности, в который они представлены
2. Для генерации данных для своего кейса необходимо перейти на [ячейку](https://colab.research.google.com/drive/17Ks3IwdBWbquXLDvQ56HsnzCY3lOqgxE#scrollTo=zqgbRKu1HQZO&line=1&uniqifier=1) и ввести свои поля

### 1. Установка библиотек и импорт модулей

In [1]:
!pip install Faker

import random
import datetime
from faker import Faker
from google.colab import files

from faker.providers.address.ru_RU import Provider

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.8/1.8 MB 7.5 MB/s eta 0:00:00


### 2. Формирование функций генерации

In [2]:
fake = Faker('ru_RU')
fake.add_provider(Provider)

def get_fake_type(*, type):
  start_date = datetime.date(2023, 1, 1)
  end_date = datetime.date(2023, 12, 31)
  products = ['яблоки желтые', 'малина', 'вода', 'хлеб белый','хлеб серый', 'креветки', 'форель', 'апельсины', 'кета','курица','яйцо перепелиное','яйцо куриное','лаваш',
            'булка сдобная','булка сахарная','помидоры бакинские','помидоры чери','огурцы','перец сладкий','перец острый','перец болгарский','мандарины','укроп свежий',
            'укроп сушеный','клубника свежая','клубника мороженная','мороженое','картошка','морковь', 'свекла','пангасиус','семга','кальмар замороженный','горошек зеленый',
            'смородина черная','смородина красная','соль поваренная пищевая йодированная','чай черный байховый','чай зеленый','чай красный','кофе','кофе с молоком','какао',
            'молоко','кефир','сыр с плесенью','сыр плавленый','сыр твердый','сыр мягкий','яблоки красные','яблоки зеленые','яблоки сушеные','икра красная',
            'икра черная','икра заморская баклажанная','масло сливочное','масло оливковое','масло подсолнечное','масло кокосовое','орех грецкий','орех бразильский',
            'лист лавровый','куркума','кукуруза','печенье сладкое','пряники сдобные','тесто слоеное','варенц','ряженка','снежок','шоколад молочный']
  if type == 'name':
    return f"'{fake.name()}'"
  elif type == 'first_name':
    return f"'{fake.first_name()}'"
  elif type == 'last_name':
    return f"'{fake.last_name()}'"
  elif type == 'middle_name':
    return f"'{fake.middle_name()}'"
  elif type == 'email':
    return f"'{fake.email()}'"
  elif type == 'phone':
    return f"'{fake.phone_number()}'"
  elif type == 'company':
    return f"'{fake.company()}'"
  elif type == 'address':
    return f"'{fake.address()}'"
  elif type.startswith('fk_int'):
    return fake.random_int(min=int(str.split(type, '_')[-2]), max=int(str.split(type, '_')[-1]))
  elif type == 'int':
    return fake.random_int(min=1, max=100)
  elif type == 'float':
    return round(fake.pyfloat(right_digits=2, min_value=1, max_value=10000), 2)
  elif type == 'date':
    return f"'{(start_date + datetime.timedelta(days=random.randint(0, (end_date - start_date).days))).strftime('%Y-%m-%d')}'"
  elif type == 'datetime':
    return f"'{fake.date_time()}'"
  elif type == 'product':
    return f"'{random.choice(products)}'"
  elif type == 'license_plate':
    return f"'{fake.license_plate()}'"
  elif type == 'chars':
    return f"'{fake.word()}'"
  elif str(type).startswith("fix_char"):
    custom_chars = str(type).split("_")[-1]
    char_list = []
    for i in custom_chars:
      char_list.append(i)
    return f"'{random.choice(char_list)}'"
  else:
    return 'Incorrect type'

def generate_rows(*, row_number, table_name, column_names):
  query = "";
  columns_str = ', '.join(f'"{column}"' for column in column_names.keys())
  for i in range(row_number):
    values = list()
    values.append(i+1)
    for k, v in column_names.items():
      if v != 'id':
        values.append(get_fake_type(type=v))
    values_str = ', '.join(str(value) for value in values)
    insert_query = f'INSERT INTO "{table_name}" ({columns_str}) VALUES ({values_str});\n'
    query += insert_query
  return query


def save_to_file(*, row_number, table_name, column_names):
  filename = f'{table_name}.sql'
  with open(filename, 'w') as f:
    generated_rows = generate_rows(row_number=row_number, table_name=table_name, column_names=column_names)
    f.write(generated_rows)
  f.close()
  files.download(filename)

### 3. Заполнение переменных

> Название таблицы

In [3]:
table_name = 'customer'

> Количество строк для генерации

In [4]:
k = 5

> Столбцы таблицы

Правила оформления:
1. Составляется ввиде словаря {"k":"v"}, где "k" - название поля, "v" - тип поля.
2. Обязательно наличие поля с типом "id"
3. Тип поля может быть одним из:


* name - ФИО, например Иван Иванович Иванов
* first_name - имя, например Иван
* middle_name - отчество, например Иванович
* last_name - фамилия, например, Иванов
* email - почта, например 123@example.org
* phone - телефон, например 89999011121 или +7 (999) 023-4444
* company - наименование компаний, например ОАО Ромашка
* address - адрес, например с. Владивосток, алл. Запорожская, д. 1 к. 1, 111111
* product - наименование продукта, например шоколад молочный
* date - дата в диапазоне от 2023-01-01 до 2023-12-31
* datetime - дата и время в формате 2018-11-25 16:47:44.613686
* float - число с плавающей точкой в диапазоне от 1 до 10000, например 78.73
* int - целое число в диапазоне от 1 до 100, например 1001
* fk_int_0_100 - поле FOREIGN KEY, где ключ, на который ссылается должен быть типа int. При этом диапазон значений (0_100) может быть изменен в соответсвии с вашим кейсом. Например, fk_int_1_5 или fk_int_0_34568
* license_plate - номер автомобиля в формате B1758 06
* chars - рандомное слово, например встать
* fix_char_[ваши символы] - будет выбирать рандомно один из переданных символов, например fix_char_ABC будет отдавать либо A либо B либо C

4. Если тип поля введен некорректно, то программа сгенерирует значение 'Incorrect type'



In [5]:
column_names = {'id':'id', 'name':'name', 'phone': 'phone', 'state': 'fk_int_0_5', 'cmp': 'company', 'addr': 'address', 'price': 'float', 'dt':'date', 'e':'email', 'products': 'product', 'quantity': 'int', 'license_plate': 'license_plate', 'datetime': 'datetime', 'first_name': 'first_name', 'middle_name': 'middle_name', 'last_name': 'last_name', 'chars': 'chars', 'status_char': 'fix_char_ABC'}

### 4. Пример генерации записей с выводом на экран

In [7]:
q = generate_rows(row_number=k, table_name=table_name, column_names=column_names)
print(q)

INSERT INTO "customer" ("id", "name", "phone", "state", "cmp", "addr", "price", "dt", "e", "products", "quantity", "license_plate", "datetime", "first_name", "middle_name", "last_name", "chars", "status_char") VALUES (1, 'Степанов Модест Харитонович', '+7 (867) 263-5461', 0, 'РАО «Горбачева Воронцов»', 'клх Видное, ул. Фабричная, д. 28, 149470', 9948.73, '2023-10-17', 'viktorin1974@example.net', 'лаваш', 89, 'KС6906 14', '2000-09-17 19:00:50.743173', 'Аполлинарий', 'Иосипович', 'Федотов', 'танцевать', 'A');
INSERT INTO "customer" ("id", "name", "phone", "state", "cmp", "addr", "price", "dt", "e", "products", "quantity", "license_plate", "datetime", "first_name", "middle_name", "last_name", "chars", "status_char") VALUES (2, 'Елизавета Олеговна Турова', '8 559 288 6133', 0, 'Гаврилова Групп', 'к. Малгобек, алл. Хуторская, д. 353 стр. 9, 840746', 9148.12, '2023-04-02', 'ddrozdova@example.org', 'семга', 66, 'AО2408 164', '1995-07-28 13:52:35.654013', 'Анна', 'Алексеевна', 'Исаева', 'юный'

### 5. Пример с генерацией записей в файл

In [ ]:
save_to_file(row_number=k, table_name=table_name, column_names=column_names)

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>